# Predicting Spam/Non-spam

* * *

In [1]:
# import required libraries
import pandas as pd
import string
from nltk.corpus import stopwords

In [2]:
#Get the spam data collection 
spam_data_collection= pd.read_csv('E:\\Melita\\Python\\Assignment_01\\NLP-Assignment 01.5\\SpamCollection.txt', sep='\t', 
                                  names=['response','message'])

In [3]:
#view first five records
spam_data_collection.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# view more information about data using describe method
spam_data_collection.describe()


,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [5]:
#view response using groupby and desribe method
spam_data_collection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [6]:
#Verify length of the messages and also add it as a new column 
spam_data_collection['length']=spam_data_collection['message'].apply(len)

In [7]:
#view first five messages with length
spam_data_collection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [8]:
#define a function to get rid of stopwords present in the messages
def remove_stopwords(mess):
    # check character to see if there are punctuations
    remove_punctuations= [char for char in mess if char not in string.punctuation]
    # from sentence
    remove_punctuation=''.join(remove_punctuations)
    #eliminate stopwords
    return [word for word in remove_punctuation.split() if word.lower() not in stopwords.words('english')]

In [9]:
#verify the function is working
spam_data_collection['message'].head().apply(remove_stopwords)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [10]:
#start text processing with vectorizer 
#import required library
from sklearn.feature_extraction.text import CountVectorizer
# initiate the vectorizer
#vectorizer= CountVectorizer()

In [11]:
#use bag of words by applying the function and fit the data into it
bag_of_words= CountVectorizer(analyzer=remove_stopwords).fit(spam_data_collection['message'])

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [12]:
#print length of bag of words stored in the vocabulary_ attribute
print len(bag_of_words.vocabulary_)

11425


In [13]:
#store the word for messages using transform method
bag_of_words_transformed= bag_of_words.transform(spam_data_collection['message'])


F:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [14]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
# import Tfidf 
from sklearn.feature_extraction.text import TfidfTransformer
#perform tfidf
tfidf_transformation= TfidfTransformer().fit(bag_of_words_transformed)

In [15]:
#print shape of the tfidf 
message_tfidf= tfidf_transformation.transform(bag_of_words_transformed)
message_tfidf.shape

(5572, 11425)

In [16]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
# import multinomialNB
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(message_tfidf,spam_data_collection['response'])

In [17]:
#check model for the predicted and expected value say for message#2 and message#5
message= spam_data_collection['message'][4]
bag_of_words_for_messages=bag_of_words.transform([message])
tfidf= tfidf_transformation.transform(bag_of_words_for_messages)

print 'predicted', spam_detect_model.predict(tfidf)[0]
print 'expected', spam_data_collection.response[4]

predicted ham
expected ham
